In [2]:
import numpy as np  
import pandas as pd 

/kaggle/input/full-sber-2-0/train_ai_comp_final_dp.parquet
/kaggle/input/full-sber-2-0/train_2_ai_comp_final_dp.parquet
/kaggle/input/full-sber-2-0/test_sber.parquet


In [3]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
import optuna
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [4]:
df = pd.read_parquet("/kaggle/input/full-sber-2-0/train_2_ai_comp_final_dp.parquet")
y = df["target"]
X = df.drop(columns = ["sample_ml_new", "target", "id"])

In [12]:
y.shape

(519615,)

In [4]:
test = pd.read_parquet("/kaggle/input/full-sber-2-0/test_sber.parquet")
test1 = test.drop(columns = ["sample_ml_new", "id"])
test1.shape

(173433, 1076)

In [5]:
scaler = StandardScaler()
scaler.fit(X)
scaler.mean_
X_1 = scaler.transform(X)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/conda/lib/python3.10/site-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/opt/conda/lib/python3.10/site-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [ ]:
def objective(trial):
    # Определение гиперпараметров
    params = {
        'objective': 'binary', # Или 'multiclass' для многоклассовой классификации
        'metric': 'binary_logloss', # Или 'multi_logloss' для многоклассовой классификации
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-8, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_uniform('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 100.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
    }
    
    gbm = lgb.LGBMClassifier(**params, device='gpu')
    # Обучение модели
    gbm.fit(X_train, y_train)
    
    pred = gbm.predict_proba(X_test)
    pred = pred[:, 1]
    auc = roc_auc_score(y_test, pred)
    
    return auc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

[I 2024-03-13 16:19:07,536] A new study created in memory with name: no-name-022e889b-493b-4e8a-8c50-dc217b2acff5
/tmp/ipykernel_34/2138691126.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-8, 1.0),
/tmp/ipykernel_34/2138691126.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.1, 1.0),
/tmp/ipykernel_34/2138691126.py:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.

[LightGBM] [Info] Number of positive: 14830, number of negative: 400862
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 112018
[LightGBM] [Info] Number of data points in the train set: 415692, number of used features: 958
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 447 dense feature groups (177.60 MB) transferred to GPU in 0.117503 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.035675 -> initscore=-3.296965
[LightGBM] [Info] Start training from score -3.296965


[I 2024-03-13 16:20:10,014] Trial 0 finished with value: 0.7287213826389651 and parameters: {'num_leaves': 96, 'learning_rate': 0.010236974264967995, 'n_estimators': 608, 'max_depth': 1, 'min_child_samples': 79, 'subsample': 0.8939198015517199, 'colsample_bytree': 0.8440278145761243, 'reg_lambda': 1.2649120492988723e-08, 'reg_alpha': 0.04490334286974734}. Best is trial 0 with value: 0.7287213826389651.
/tmp/ipykernel_34/2138691126.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-8, 1.0),
/tmp/ipykernel_34/2138691126.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.1, 1.0

[LightGBM] [Info] Number of positive: 14830, number of negative: 400862
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 112018
[LightGBM] [Info] Number of data points in the train set: 415692, number of used features: 958
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 447 dense feature groups (177.60 MB) transferred to GPU in 0.126771 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.035675 -> initscore=-3.296965
[LightGBM] [Info] Start training from score -3.296965
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

Exception ignored on calling ctypes callback function: <function _log_callback at 0x7e9d816e00d0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py", line 224, in _log_callback
    def _log_callback(msg: bytes) -> None:
KeyboardInterrupt: 


...

No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

Exception ignored on calling ctypes callback function: <function _log_callback at 0x7e9d816e00d0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py", line 224, in _log_callback
    def _log_callback(msg: bytes) -> None:
KeyboardInterrupt: 


No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


Exception ignored on calling ctypes callback function: <function _log_callback at 0x7e9d816e00d0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py", line 224, in _log_callback
    def _log_callback(msg: bytes) -> None:
KeyboardInterrupt: 


No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [9]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X_1), np.array(y), test_size=0.2, stratify = y, random_state=42)

In [ ]:
#Конечная модель, используемая для оценки на тесте
model = lgb.LGBMClassifier(device='gpu',
                          boosting_type='gbdt', # reg_lambda = 0.9, 
                          max_depth=-1,
                          learning_rate=0.1,
                          num_leaves=27,
                          n_estimators=120,
                          objective='binary',
                          class_weight={1:20.5, 0:1},
                          random_state=160504)
model.fit(X_1, y)

In [ ]:
#def encoding(result, t):
#    y_pred = []
#    for p in result:
#        if p >= t:
#            y_pred.append(1)
#        else:
#           y_pred.append(0)
#        
#    return y_pred

#fmax_result = []
#precision_result  = []
#recall_result = []

pred = model.predict_proba(X_test)
#pred_1 = list(map(lambda x: x[0], pred))
#for t in range(1, 101):
#    t = t / 100
#    y_pred_1 = encoding(pred_1, t)
#    f1_max = f1_score(y_test, y_pred_1)
#    pr = precision_score(y_test, y_pred_1)
#    rc = recall_score(y_test, y_pred_1)
#    fmax_result.append(f1_max)
#    precision_result.append(pr)
#    recall_result.append(rc)

#pred_binary_fmax = max(fmax_result)
#index_fmax = fmax_result.index(max(fmax_result))
#pred_binary_pr = precision_result[index_fmax]
#pred_binary_rc = recall_result[index_fmax]


pred = pred[:, 1]
pred_binary = (pred >= 0.65)

print("F1_SCORE", f1_score(y_test, pred_binary))
print("PRECISION", precision_score(y_test, pred_binary))
print("RECALL", recall_score(y_test, pred_binary))
print("ROC_AUC", roc_auc_score(y_test, pred))

print('INDEX', index_fmax)

In [ ]:
#feature_importance = pd.Series(model.feature_importances_, index=X.columns)
#plt.figure(figsize=(10, 6))
#feature_importance.nlargest(25).plot(kind='barh')
#plt.title('Feature Importance')
#plt.show()

In [ ]:
scaler = StandardScaler()
scaler.fit(test1)
scaler.mean_
test1 = scaler.transform(test1)

pred = model.predict_proba(test1)

pred = pred[:, 1]
pred_binary = (pred >= 0.65).astype(int)

In [ ]:
d = {'id': test.id, 'target_bin': pred_binary, 'target_prob': pred}

In [ ]:
submission = pd.DataFrame(d)
submission.head(3)

In [ ]:
submission.to_csv("submission.csv", index=False)

In [ ]:
submission.target_bin.value_counts()